# **Group Study Season 1-1: Deep Graph Learning From Scratch**

#### The materaials for Group Study in Perception Lab Durham University
#### This season, we are going to focus on one of the amazing technologies in the deep learning era, which may be used in everyone's future research. Our main goal is to help everyone understand this technique not only from the aspect of theory but also from the codebase.










> Today's takeaway (27/05/2022)


*   Simple/Conventional/Basic Pytorch Deep Learning framework
*   Simpe Graph Neural Network
*   Traning with mnist

> Some useful resources:

*   Highly recommended: https://distill.pub/2021/gnn-intro/ (~ 2 hour)
*   https://tkipf.github.io/graph-convolutional-networks/ (father of graph in my opinion)



# Graph Neural Network

### Graph representation

Before starting the discussion of specific neural network operations on graphs, we should consider how to represent a graph. Mathematically, a graph $\mathcal{G}$ is defined as a tuple of a set of nodes/vertices $V$, and a set of edges/links $E$: $\mathcal{G}=(V,E)$. Each edge is a pair of two vertices, and represents a connection between them.

The vertices are $V=\{1,2,3,4\}$, and edges $E=\{(1,2), (2,3), (2,4), (3,4)\}$. Note that for simplicity, we assume the graph to be undirected and hence don't add mirrored pairs like $(2,1)$. In application, vertices and edge can often have specific attributes, and edges can even be directed. The question is how we could represent this diversity in an efficient way for matrix operations. Usually, for the edges, we decide between two variants: an adjacency matrix, or a list of paired vertex indices. 

The **adjacency matrix** $A$ is a square matrix whose elements indicate whether pairs of vertices are adjacent, i.e. connected, or not. In the simplest case, $A_{ij}$ is 1 if there is a connection from node $i$ to $j$, and otherwise 0. If we have edge attributes or different categories of edges in a graph, this information can be added to the matrix as well. For an undirected graph, keep in mind that $A$ is a symmetric matrix ($A_{ij}=A_{ji}$). For the example graph above, we have the following adjacency matrix:

$$
A = \begin{bmatrix}
    0 & 1 & 0 & 0\\
    1 & 0 & 1 & 1\\
    0 & 1 & 0 & 1\\
    0 & 1 & 1 & 0
\end{bmatrix}
$$

While expressing a graph as a list of edges is more efficient in terms of memory and (possibly) computation, using an adjacency matrix is more intuitive and simpler to implement. In our implementations below, we will rely on the adjacency matrix to keep the code simple. However, common libraries use edge lists, which we will discuss later more.
Alternatively, we could also use the list of edges to define a sparse adjacency matrix with which we can work as if it was a dense matrix, but allows more memory-efficient operations. PyTorch supports this with the sub-package `torch.sparse` ([documentation](https://pytorch.org/docs/stable/sparse.html)) which is however still in a beta-stage (API might change in future).
Below we show some simple examples.

In [ ]:
import os
import json
import math
import numpy as np
import time
import argparse
## Imports for plotting
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('svg', 'pdf') # For export
from matplotlib.colors import to_rgb
import matplotlib
from scipy.spatial.distance import cdist
matplotlib.rcParams['lines.linewidth'] = 2.0
import seaborn as sns
sns.reset_orig()
sns.set()

## Progress bar
from tqdm.notebook import tqdm

## PyTorch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data
import torch.optim as optim
# Torchvision
import torchvision
from torchvision.datasets import CIFAR10
# from torchvision import transforms
from torchvision import datasets, transforms

In [ ]:
# create some nodes, randomly give 2-d features to each of them, we have 4 nodes here [4,2]
X = np.matrix([
            [i, -i]
            for i in range(4)
        ], dtype=float)
X

matrix([[ 0.,  0.],
        [ 1., -1.],
        [ 2., -2.],
        [ 3., -3.]])

In [ ]:
### Adjacency Matrix: the coonection/edge of a graph [4,4]
A = np.matrix([
    [0, 1, 0, 0],
    [0, 0, 1, 1], 
    [0, 1, 0, 0],
    [1, 0, 1, 0]],
    dtype=float
)

# row-number == node-number
# 0: No connection/edge, 1: has connection/edge
# [4,4] pair-wise connectivity

In [ ]:
# The representation of each node (each row) is now the sum of the features of its neighbors
A * X

matrix([[ 1., -1.],
        [ 5., -5.],
        [ 1., -1.],
        [ 2., -2.]])

#### Problems:


*   The aggregated representation of a node does not contain its own features! The representation is a feature aggregation of neighboring nodes, so only nodes with self-loops will include their own features in this aggregation.

*   A node with a large degree will have a larger value in its feature representation, and a node with a small degree will have a smaller value. This can lead to vanishing gradients or exploding gradients, and also affects stochastic gradient descent algorithms (stochastic gradient descent algorithms are often used to train such networks, and the scale (or range of values) of each input feature are sensitive).



```
1. For undirected graphs, the degree of a vertex v is the number of edges associated with v.
2. For a directed graph, the number of arcs headed by vertex v is called the in-degree of v
3. The number of arcs ending at vertex v is called the out-degree of v
```


we are showing how to alleviate these two problems below.




In [ ]:
# ADD self-loop
I = np.matrix(np.eye(A.shape[0]))
A_hat = A + I
A_hat * X
# Now, since each node is its own neighbor, each node also includes its own features in the process of summing the features of neighboring nodes!

matrix([[ 1., -1.],
        [ 6., -6.],
        [ 3., -3.],
        [ 5., -5.]])

Normalize the graph feature representation
The feature representation is normalized by the degree of the nodes by transforming the adjacency matrix A by multiplying it by the inverse of the degree matrix D. Therefore, our simplified propagation rules are as follows:

f(X, A) = D⁻¹AX

Let's see what happened. We first compute the degree matrix of the nodes.

In [ ]:
# Degree
D = np.array(np.sum(A, axis=0))[0]
D = np.matrix(np.diag(D))
D

matrix([[1., 0., 0., 0.],
        [0., 2., 0., 0.],
        [0., 0., 2., 0.],
        [0., 0., 0., 1.]])

In [ ]:
D**-1 * A

matrix([[0. , 1. , 0. , 0. ],
        [0. , 0. , 0.5, 0.5],
        [0. , 0.5, 0. , 0. ],
        [1. , 0. , 1. , 0. ]])

In [ ]:
# Normalized output, now comparing it with previous output without normalization
D**-1 * A * X

matrix([[ 1. , -1. ],
        [ 2.5, -2.5],
        [ 0.5, -0.5],
        [ 2. , -2. ]])

Then, we are trying to add weight matrix and activation function

In [ ]:
# example 1 the activation function has the same size of it and can be placed simply
W = np.matrix([
             [1, -1],
             [-1, 1]
         ])
D**-1 * A * X * W

matrix([[ 2., -2.],
        [ 5., -5.],
        [ 1., -1.],
        [ 4., -4.]])

In [ ]:
# example 2 the activation function has the same size of it and can be placed simply
W = np.matrix([
             [1],
             [-1]
         ])
D**-1 * A * X * W

matrix([[2.],
        [5.],
        [1.],
        [4.]])

Until now, we obtained a very classical and important formulation of GNN/GCN:

$$H^{(l+1)} = \sigma\left(\hat{D}^{-1/2}\hat{A}\hat{D}^{-1/2}H^{(l)}W^{(l)}\right)$$



> Following, we try a more mature examples that we are trying to make the graph as a layer via pytorch





In [ ]:
class GCNLayer(nn.Module):
    
    def __init__(self, c_in, c_out):
        super().__init__()
        self.projection = nn.Linear(c_in, c_out)

    def forward(self, node_feats, adj_matrix):
        """
        Inputs:
            node_feats - Tensor with node features of shape [batch_size, num_nodes, c_in]
            adj_matrix - Batch of adjacency matrices of the graph. If there is an edge from i to j, adj_matrix[b,i,j]=1 else 0.
                         Supports directed edges by non-symmetric matrices. Assumes to already have added the identity connections. 
                         Shape: [batch_size, num_nodes, num_nodes]
        """
        # Num neighbours = number of incoming edges
        num_neighbours = adj_matrix.sum(dim=-1, keepdims=True)
        print('num_neighbours shape: {}'.format(num_neighbours.shape))
        print('Input node feats shape: {}'.format(num_neighbours.shape))
        node_feats = self.projection(node_feats)
        print('Learnt node feats shape: {}'.format(num_neighbours.shape))
        node_feats = torch.bmm(adj_matrix, node_feats)
        print('Obtained graph feats shape: {}'.format(num_neighbours.shape))
        # Simple normaliztion
        node_feats = node_feats / num_neighbours
        return node_feats

create a sample to forward try the layer

In [ ]:
node_feats = torch.arange(8, dtype=torch.float32).view(1, 4, 2)
adj_matrix = torch.Tensor([[[1, 1, 0, 0],
                            [1, 1, 1, 1],
                            [0, 1, 1, 1],
                            [0, 1, 1, 1]]])

print("Node features:\n", node_feats)
print("\nAdjacency matrix:\n", adj_matrix)

Node features:
 tensor([[[0., 1.],
         [2., 3.],
         [4., 5.],
         [6., 7.]]])

Adjacency matrix:
 tensor([[[1., 1., 0., 0.],
         [1., 1., 1., 1.],
         [0., 1., 1., 1.],
         [0., 1., 1., 1.]]])


In [ ]:
layer = GCNLayer(c_in=2, c_out=2)
with torch.no_grad():
    out_feats = layer(node_feats, adj_matrix)

print("Adjacency matrix", adj_matrix)
print("Input features", node_feats)
print("Output features", out_feats)

num_neighbours shape: torch.Size([1, 4, 1])
Input node feats shape: torch.Size([1, 4, 1])
Learnt node feats shape: torch.Size([1, 4, 1])
Obtained graph feats shape: torch.Size([1, 4, 1])
Adjacency matrix tensor([[[1., 1., 0., 0.],
         [1., 1., 1., 1.],
         [0., 1., 1., 1.],
         [0., 1., 1., 1.]]])
Input features tensor([[[0., 1.],
         [2., 3.],
         [4., 5.],
         [6., 7.]]])
Output features tensor([[[0.6454, 0.4224],
         [1.9260, 0.7979],
         [2.5662, 0.9857],
         [2.5662, 0.9857]]])


# A simple deep learning project (MNIST) using a graph network



---



> Set neccessary parameters



In [ ]:
batch_size = 4
device = 'cpu'
lr = 1e-3
epochs = 1

---


> Loading the data






In [ ]:
# loading the data
train_loader = torch.utils.data.DataLoader(datasets.MNIST('./data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)

test_loader = torch.utils.data.DataLoader(datasets.MNIST('./data', train=False, transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
    ])),
    batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
# simple test of dataloader
for batch_idx, (data, target) in enumerate(train_loader):
  print(data.shape)
  print(target.shape)
  break

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


torch.Size([4, 1, 28, 28])
torch.Size([4])




---



> Create the model



In [ ]:
class GraphNet(nn.Module):
    def __init__(self, image_size = 28, pred_edge = False):
        super(GraphNet, self).__init__()
        self.pred_edge = pred_edge
        N = image_size ** 2 # Number of pixels in the image
        self.fc = nn.Linear(N, 10, bias = False)
        # Create the adjacency matrix of size (N X N)
        if pred_edge:
            # Learn the adjacency matrix (learn to predict the edge between any pair of pixels)
            col, row = np.meshgrid(np.arange(image_size), np.arange(image_size)) # (28 x 28) Explanation: https://www.geeksforgeeks.org/numpy-meshgrid-function/
            # 28*28 pair-wise pixel
            coord = np.stack((col, row), axis = 2).reshape(-1, 2)  # (784 x 2)
            coord_normalized = (coord - np.mean(coord, axis = 0)) / (np.std(coord, axis = 0) + 1e-5) # Normalize the matrix
            coord_normalized = torch.from_numpy(coord_normalized).float() # (784 x 2)
            adjacency_matrix = torch.cat((coord_normalized.unsqueeze(0).repeat(N, 1,  1),
                                    coord_normalized.unsqueeze(1).repeat(1, N, 1)), dim=2) # (784 x 784 x 4)
            self.pred_edge_fc = nn.Sequential(nn.Linear(4, 64),
                                              nn.ReLU(), 
                                              nn.Linear(64, 1),
                                              nn.Tanh())
            self.register_buffer('adjacency_matrix', adjacency_matrix) # not to be considered a model paramater that is updated during training
        else:
            # Use a pre-computed adjacency matrix
            A = self.precompute_adjacency_images(image_size)
            self.register_buffer('A', A) # not to be considered a model paramater that is updated during training

    def forward(self, x):
        '''
        x: image (batch_size x 1 x image_width x image_height)
        each image is denoted as a graph
        '''
        B = x.size(0) # 64
        if self.pred_edge:
            self.A = self.pred_edge_fc(self.adjacency_matrix).squeeze() # (784 x 784) --> predicted edge map

        avg_neighbor_features = (torch.bmm(self.A.unsqueeze(0).expand(B, -1, -1), 
                                            x.view(B, -1, 1)).view(B, -1)) # (64 X 784)
        return self.fc(avg_neighbor_features)

    @staticmethod
    # Static method knows nothing about the class and just deals with the parameters.
    def precompute_adjacency_images(image_size):
        print('precompute_adjacency_images')
        col, row = np.meshgrid(np.arange(image_size), np.arange(
            image_size))  # (28 x 28) Explanation: https://www.geeksforgeeks.org/numpy-meshgrid-function/
        coord = np.stack((col, row), axis=2).reshape(-1, 2) / image_size  # (784 x 2) --> normalize
        dist = cdist(coord, coord)  # compute distance between every pair of pixels
        sigma = 0.05 * np.pi  # width of the Gaussian (can be a hyperparameter while training a model)
        A = np.exp(-dist / sigma ** 2)  # adjacency matrix of spatial similarity
        A[A < 0.01] = 0  # suppress values less than 0.01
        A = torch.from_numpy(A).float()

        # Normalization as per (Kipf & Welling, ICLR 2017)
        D = A.sum(1)  # nodes degree (N,)
        D_hat = (D + 1e-5) ** (-0.5)
        A_hat = D_hat.view(-1, 1) * A * D_hat.view(1, -1)  # N,N

        # Some additional trick I found to be useful
        A_hat[A_hat > 0.0001] = A_hat[A_hat > 0.0001] - 0.2

        print(A_hat[:10, :10])
        return A_hat

precompute_adjacency_images
tensor([[ 0.3400, -0.0852, -0.1736, -0.1938,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000],
        [-0.0852,  0.2413, -0.0987, -0.1763, -0.1944,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000],
        [-0.1736, -0.0987,  0.2207, -0.1015, -0.1768, -0.1946,  0.0000,  0.0000,
          0.0000,  0.0000],
        [-0.1938, -0.1763, -0.1015,  0.2166, -0.1020, -0.1770, -0.1946,  0.0000,
          0.0000,  0.0000],
        [ 0.0000, -0.1944, -0.1768, -0.1020,  0.2166, -0.1020, -0.1770, -0.1946,
          0.0000,  0.0000],
        [ 0.0000,  0.0000, -0.1946, -0.1770, -0.1020,  0.2166, -0.1020, -0.1770,
         -0.1946,  0.0000],
        [ 0.0000,  0.0000,  0.0000, -0.1946, -0.1770, -0.1020,  0.2166, -0.1020,
         -0.1770, -0.1946],
        [ 0.0000,  0.0000,  0.0000,  0.0000, -0.1946, -0.1770, -0.1020,  0.2166,
         -0.1020, -0.1770],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000, -0.1946, -0.1770, -0.1020,
          0.2166, -



---



> Define the training materials



In [ ]:
model = GraphNet()
model = model.to(device)
optimizer = optim.SGD(model.parameters(), lr=lr, weight_decay=1e-4)

precompute_adjacency_images
tensor([[ 0.3400, -0.0852, -0.1736, -0.1938,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000],
        [-0.0852,  0.2413, -0.0987, -0.1763, -0.1944,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000],
        [-0.1736, -0.0987,  0.2207, -0.1015, -0.1768, -0.1946,  0.0000,  0.0000,
          0.0000,  0.0000],
        [-0.1938, -0.1763, -0.1015,  0.2166, -0.1020, -0.1770, -0.1946,  0.0000,
          0.0000,  0.0000],
        [ 0.0000, -0.1944, -0.1768, -0.1020,  0.2166, -0.1020, -0.1770, -0.1946,
          0.0000,  0.0000],
        [ 0.0000,  0.0000, -0.1946, -0.1770, -0.1020,  0.2166, -0.1020, -0.1770,
         -0.1946,  0.0000],
        [ 0.0000,  0.0000,  0.0000, -0.1946, -0.1770, -0.1020,  0.2166, -0.1020,
         -0.1770, -0.1946],
        [ 0.0000,  0.0000,  0.0000,  0.0000, -0.1946, -0.1770, -0.1020,  0.2166,
         -0.1020, -0.1770],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000, -0.1946, -0.1770, -0.1020,
          0.2166, -



---


> Define the training function




In [ ]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        # Cross entropy loss
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 1000 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                       100. * batch_idx / len(train_loader), loss.item()))



---



> Define the testing function



In [ ]:
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.cross_entropy(output, target, reduction='sum').item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
    test_loss /= len(test_loader.dataset)
    print(
        '\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
            test_loss, correct, len(test_loader.dataset),
            100. * correct / len(test_loader.dataset)))



---



> Start training



In [ ]:
for epoch in range(1, epochs + 1):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Train Epoch: 1 [0/60000 (0%)]	Loss: 2.607061
Train Epoch: 1 [4000/60000 (7%)]	Loss: 1.534182
Train Epoch: 1 [8000/60000 (13%)]	Loss: 0.131974
Train Epoch: 1 [12000/60000 (20%)]	Loss: 0.003211
Train Epoch: 1 [16000/60000 (27%)]	Loss: 1.335701
Train Epoch: 1 [20000/60000 (33%)]	Loss: 0.028707
Train Epoch: 1 [24000/60000 (40%)]	Loss: 0.312732
Train Epoch: 1 [28000/60000 (47%)]	Loss: 0.085027
Train Epoch: 1 [32000/60000 (53%)]	Loss: 3.886122
Train Epoch: 1 [36000/60000 (60%)]	Loss: 0.067824
Train Epoch: 1 [40000/60000 (67%)]	Loss: 0.472454
Train Epoch: 1 [44000/60000 (73%)]	Loss: 0.031215
Train Epoch: 1 [48000/60000 (80%)]	Loss: 0.135806
Train Epoch: 1 [52000/60000 (87%)]	Loss: 0.035626
Train Epoch: 1 [56000/60000 (93%)]	Loss: 1.400886

Test set: Average loss: 0.3974, Accuracy: 8907/10000 (89%)



# Next time: Graph Attention